# The COVIDNetX challenge

<img src="https://www.psycharchives.org/retrieve/096175aa-f7f2-4970-989d-d934c30b5551" alt="drawing" width="400"/>

The following is a classification challenge using the COVID-X dataset (https://github.com/lindawangg/COVID-Net/blob/master/docs/COVIDx.md).
The goal is to predict whether a person has COVID-19 or not based on chest X-RAY images.

There are two different categories: `positive` and `negative`.
`positive` means a person has COVID-19, `negative` means a person
has not COVID-19.

The metric we use is F1 (https://en.wikipedia.org/wiki/F1_score). The goal
is to maximize F1.

The data contains images with their associated labels.

# Exploratory Data Analysis

In [ ]:
import pandas as pd
from IPython.display import Image

data_dir = 'data/'
# data_dir = 'data_subset/'

Now, you can open the submision.csv file (File -> Open) file and download it!

After you download it, you can upload it to the challenge frontend.

# Baseline simple solution (Logistic Regression on the top of Resnet50 features)

In this simple baseline solution, we use the learned representation by a Resnet-50 as features
and use the logistic regression as a model (no fine-tuning here).

In [ ]:
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision
import time
import os

class CustomDataSet(Dataset):
    def __init__(self, filenames, transform=None, labels=None):
        self.filenames = filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx]).convert("RGB")
        label = self.labels[idx] if self.labels is not None else 0
        if self.transform:
          tensor_image = self.transform(image)
        return tensor_image, label

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
transform =  transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      normalize
])
classname_to_index = {"positive": 1, "negative": 0}
index_to_classname = {i:n for n, i in classname_to_index.items()}

def build_dataset(df, split):
  filenames = [os.path.join(split, name) for name in df.image]
  labels = [classname_to_index[name] for name in df.label]
  dataset = CustomDataSet(filenames, transform=transform, labels=labels)
  return dataset

df_train = pd.read_csv(data_dir+'train.csv')
df_test = pd.read_csv(data_dir+'submission_valid.csv')
train_dataset = build_dataset(df_train.sample(frac=0.1), data_dir+"train")
test_dataset = build_dataset(df_test.sample(frac=0.1), data_dir+"valid")
len(train_dataset), len(test_dataset)

In [ ]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

In [ ]:
from tqdm import tqdm
model = torchvision.models.resnet50(pretrained=True)

def extract_features(model, dataloader):
    Flist = []
    Ylist = []
    for X, Y in tqdm(dataloader):
        x = model.conv1(X)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)

        x = model.layer1(x)
        x = model.layer2(x)
        x = model.layer3(x)
        x = model.layer4(x)
        x = model.avgpool(x)
        x = x.view(x.size(0), x.size(1))
        Flist.append(x)
        Ylist.append(Y)
    F = torch.cat(Flist).detach().numpy()
    Y = torch.cat(Ylist).detach().numpy()
    return F, Y

In [ ]:
# resnet50 = torchvision.models.resnet50(pretrained=True)
# resnet50

# vgg16 = torchvision.models.vgg16(pretrained=True)
# vgg16

In [ ]:
%%time
xtrain, ytrain = extract_features(model, train_loader)
# xtest, ytest = extract_features(model, test_loader)

In [ ]:
print(xtrain.shape, ytrain.shape)
# print(xtest.shape, ytest.shape)

In [ ]:
import numpy as np
idx = np.random.choice(range(len(xtrain)), int(0.8*len(xtrain)), replace=False)
train_idx = np.zeros(len(xtrain), dtype=bool)
train_idx[idx] = True
test_idx = ~train_idx

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,class_weight="balanced")
clf.fit(xtrain[train_idx], ytrain[train_idx])

In [ ]:
# ypred_test = clf.predict(xtrain[400:])
# ypred_test = [index_to_classname[y] for y in ypred_test]
# print(ypred_test[0:10])
# df = pd.read_csv(data_dir+'submission_valid.csv')
# df["label"] = ypred_test
# df.head()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

print("############################## TRAIN ############################## ")
y_pred = clf.predict(xtrain[train_idx])
print(classification_report(ytrain[train_idx], y_pred))

print("############################## TEST ############################## ")
y_pred = clf.predict(xtrain[test_idx])
print(classification_report(ytrain[test_idx], y_pred))
# cm = confusion_matrix(ytrain[test_idx], y_pred)
# sns.heatmap(cm, cmap='plasma', annot=True);

In [ ]:
# df.to_csv("submission.csv", index=False)

Now, you can open the submision.csv file (File -> Open) file and download it!

After you download it, you can upload it to the challenge frontend.

# Baseline Solution with Resnet-50 Fine-tuning

In [ ]:
#Code adapted from https://github.com/pytorch/examples/tree/master/imagenet
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import time
import os
import shutil
# device = "cpu"
device = "cuda"

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        target = target.to(device)
        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        (acc1,) = accuracy(output, target, topk=(1,))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            progress.display(i)


def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(device)
            target = target.to(device)
            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, = accuracy(output, target, topk=(1,))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


def save_checkpoint(state, is_best, filename='models/checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'models/model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def adjust_learning_rate(optimizer, epoch, base_lr):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = base_lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, filenames, transform=None, labels=None):
        self.filenames = filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx]).convert("RGB")
        label = self.labels[idx] if self.labels is not None else 0
        if self.transform:
          tensor_image = self.transform(image)
        return tensor_image, label

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
transform =  transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      normalize
])
classname_to_index = {"positive": 1, "negative": 0}
nb_classes = len(classname_to_index)
index_to_classname = {idx:name for name, idx in classname_to_index.items()}
def build_dataset(df, split):
  filenames = [os.path.join(split, name) for name in df.image]
  labels = [classname_to_index[name] for name in df.label]
  dataset = CustomDataSet(filenames, transform=transform, labels=labels)
  return dataset


df_train_full = pd.read_csv(data_dir+'train.csv')
df_test = pd.read_csv(data_dir+'submission_valid.csv')
nb =int(len(df_train_full)*0.9)
df_train = df_train_full.iloc[0:nb]
df_valid = df_train_full.iloc[nb:]
train_dataset = build_dataset(df_train, data_dir+"train")
valid_dataset = build_dataset(df_valid, data_dir+"train")
test_dataset = build_dataset(df_test, data_dir+"valid")

In [ ]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    valid_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

In [ ]:
import torchvision
import torch.nn as nn
model = torchvision.models.resnet50(pretrained=True).to(device)
model.fc = nn.Linear(2048, nb_classes).to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
model

In [ ]:
%%time
best_acc1 = 0.0
for epoch in range(20):
    adjust_learning_rate(optimizer, epoch, base_lr=lr)
    train(train_loader, model, criterion, optimizer, epoch)
    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)

# Submission

In [ ]:
%%time
ypred_test = []
for X, _ in test_loader:
    with torch.no_grad():
        X = X.to(device)
        y = model(X)
        y = y.cpu()
        _, pred = y.max(dim=1)
    pred = pred.tolist()
    ypred_test.extend([index_to_classname[p] for p in pred])

In [ ]:
print(ypred_test[0:10])
df = pd.read_csv('submission_valid.csv')
df["label"] = ypred_test
df.head()

In [ ]:
df.to_csv("submission.csv", index=False)

Now, you can open the submision.csv file (File -> Open) file and download it!

After you download it, you can upload it to the challenge frontend.